## Installation

In [1]:
%%capture
!pip install vllm==0.7.1 evaluate==0.4.3 rouge_score==0.1.2 bitsandbytes==0.45.1

## Global Variables

In [2]:
dataset_name = (
    input(
        "Enter the name of the generated dataset during Lesson 3. Hit enter to default to our cached generated dataset."
    )
    or "pauliusztin/second_brain_course_summarization_task"
)
print(f"{dataset_name=}")
model_name = (
    input(
        "Enter the name of fine-tuned LLM during Lesson 4. Hit enter to default to our fine-tuned LLM."
    )
    or "pauliusztin/Meta-Llama-3.1-8B-Instruct-Second-Brain-Summarization"
)
print(f"{model_name=}")

Enter the name of the generated dataset during Lesson 3. Hit enter to default to our cached generated dataset.
dataset_name='pauliusztin/second_brain_course_summarization_task'
Enter the name of fine-tuned LLM during Lesson 4. Hit enter to default to our fine-tuned LLM.
model_name='pauliusztin/Meta-Llama-3.1-8B-Instruct-Second-Brain-Summarization'


In [3]:
import torch


def get_gpu_info() -> str | None:
    """Gets GPU device name if available.

    Returns:
        str | None: Name of the GPU device if available, None if no GPU is found.
    """
    if not torch.cuda.is_available():
        return None

    gpu_name = torch.cuda.get_device_properties(0).name

    return gpu_name


active_gpu_name = get_gpu_info()

print("GPU type:")
print(active_gpu_name)

GPU type:
Tesla T4


Depending on the type of GPU you are using, we pick a max evaluation sample number to avoid waiting too much to generate the answers required for evaluation.

In [4]:
if active_gpu_name and "T4" in active_gpu_name:
    max_evaluation_samples = 8
elif active_gpu_name and ("A100" in active_gpu_name or "L4" in active_gpu_name):
    max_evaluation_samples = 70
elif active_gpu_name:
    max_evaluation_samples = 8
else:
    raise ValueError("No Nvidia GPU found.")

print("--- Parameters ---")
print(f"{max_evaluation_samples=}")

--- Parameters ---
max_evaluation_samples=8


## Load Fine-tuned LLM

In [5]:
#vLLM is an open-source library designed to make large language model (LLM) inference and serving faster, cheaper, and easier.
from vllm import LLM

llm = LLM(
    model=model_name,
    max_model_len=4096,
    dtype="float16",
    quantization="bitsandbytes",
    load_format="bitsandbytes",
)

INFO 03-09 19:30:34 __init__.py:183] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

WARNING 03-09 19:30:49 config.py:2368] Casting torch.bfloat16 to torch.float16.
INFO 03-09 19:31:02 config.py:526] This model supports multiple tasks: {'score', 'generate', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
WARNING 03-09 19:31:02 config.py:605] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 03-09 19:31:04 config.py:975] MLA is not supported with bitsandbytes quantization. Disabling MLA.
INFO 03-09 19:31:04 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='pauliusztin/Meta-Llama-3.1-8B-Instruct-Second-Brain-Summarization', speculative_config=None, tokenizer='pauliusztin/Meta-Llama-3.1-8B-Instruct-Second-Brain-Summarization', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_paralle

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 03-09 19:35:48 model_runner.py:1116] Loading model weights took 5.3422 GB
WARNING 03-09 19:36:09 config.py:975] MLA is not supported with bitsandbytes quantization. Disabling MLA.
WARNING 03-09 19:36:09 config.py:975] MLA is not supported with bitsandbytes quantization. Disabling MLA.
INFO 03-09 19:36:09 worker.py:266] Memory profiling takes 20.81 seconds
INFO 03-09 19:36:09 worker.py:266] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 03-09 19:36:09 worker.py:266] model weights take 5.34GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.20GiB; the rest of the memory reserved for KV Cache is 6.68GiB.
INFO 03-09 19:36:09 executor_base.py:108] # CUDA blocks: 3418, # CPU blocks: 2048
INFO 03-09 19:36:09 executor_base.py:113] Maximum concurrency for 4096 tokens per request: 13.35x
WARNING 03-09 19:36:09 config.py:975] MLA is not supported with bitsandbytes quantization. Disabling MLA.
WARNING 03-09

Capturing CUDA graph shapes: 100%|██████████| 35/35 [01:35<00:00,  2.72s/it]

INFO 03-09 19:37:47 model_runner.py:1563] Graph capturing finished in 95 secs, took 0.71 GiB
INFO 03-09 19:37:47 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 119.72 seconds


## Prepare Input Samples

In [6]:
from datasets import load_dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
{}

### Response:
{}"""


def format_sample(sample: dict) -> str:
    return alpaca_prompt.format(sample["instruction"], "")

In [7]:
dataset = load_dataset(dataset_name, split="test")
dataset = dataset.select(range(max_evaluation_samples))

README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/955k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/575 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/72 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/72 [00:00<?, ? examples/s]

In [8]:
len(dataset)

8

In [9]:
dataset[0]["instruction"][:1000]

'# Notes\n\n\n\n<child_page>\n# Design Patterns\n\n# Training code\n\nThe most natural way of splitting the training code:\n- Dataset\n- DatasetLoader\n- Model\n- ModelFactory\n- Trainer (takes in the dataset and model)\n- Evaluator\n\n# Serving code\n\n[Infrastructure]Model (takes in the trained model)\n\t- register\n\t- deploy\n</child_page>\n\n\n---\n\n# Resources [Community]\n\n# Resources [Science]\n\n# Tools'

In [10]:
dataset[0]["answer"][:1000]

'```markdown\n# TL;DR Summary\n\n## Design Patterns\n- **Training Code Structure**: Key components include Dataset, DatasetLoader, Model, ModelFactory, Trainer, and Evaluator.\n- **Serving Code**: Infrastructure for Model registration and deployment.\n\n## Tags\n- Generative AI\n- LLMs\n```'

In [11]:
dataset = dataset.map(lambda sample: {"prompt": format_sample(sample)})

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [12]:
dataset[0]

{'instruction': '# Notes\n\n\n\n<child_page>\n# Design Patterns\n\n# Training code\n\nThe most natural way of splitting the training code:\n- Dataset\n- DatasetLoader\n- Model\n- ModelFactory\n- Trainer (takes in the dataset and model)\n- Evaluator\n\n# Serving code\n\n[Infrastructure]Model (takes in the trained model)\n\t- register\n\t- deploy\n</child_page>\n\n\n---\n\n# Resources [Community]\n\n# Resources [Science]\n\n# Tools',
 'answer': '```markdown\n# TL;DR Summary\n\n## Design Patterns\n- **Training Code Structure**: Key components include Dataset, DatasetLoader, Model, ModelFactory, Trainer, and Evaluator.\n- **Serving Code**: Infrastructure for Model registration and deployment.\n\n## Tags\n- Generative AI\n- LLMs\n```',
 'prompt': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a helpful assistant specialized in summarizing documents. Ge

In [13]:
dataset["prompt"][0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights\n\n### Input:\n# Notes\n\n\n\n<child_page>\n# Design Patterns\n\n# Training code\n\nThe most natural way of splitting the training code:\n- Dataset\n- DatasetLoader\n- Model\n- ModelFactory\n- Trainer (takes in the dataset and model)\n- Evaluator\n\n# Serving code\n\n[Infrastructure]Model (takes in the trained model)\n\t- register\n\t- deploy\n</child_page>\n\n\n---\n\n# Resources [Community]\n\n# Resources [Science]\n\n# Tools\n\n### Response:\n'

## Generate Answers

In [14]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.0, top_p=0.95, min_p=0.05, max_tokens=4096
)
predictions = llm.generate(dataset["prompt"], sampling_params)

Processed prompts:   0%|          | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 03-09 19:41:02 scheduler.py:947] Input prompt (57245 tokens) is too long and exceeds limit of 4096
WARNING 03-09 19:41:02 scheduler.py:947] Input prompt (24835 tokens) is too long and exceeds limit of 4096
WARNING 03-09 19:41:02 scheduler.py:947] Input prompt (10581 tokens) is too long and exceeds limit of 4096
WARNING 03-09 19:41:02 scheduler.py:947] Input prompt (15903 tokens) is too long and exceeds limit of 4096
WARNING 03-09 19:41:11 scheduler.py:947] Input prompt (9221 tokens) is too long and exceeds limit of 4096


Processed prompts: 100%|██████████| 8/8 [01:22<00:00, 10.26s/it, est. speed input: 1489.47 toks/s, output: 3.50 toks/s]


In [15]:
predictions[0].outputs[0].text

'```markdown\n# TL;DR Summary\n\n**Design Patterns:**\n\n  * Split training code into:\n    * Dataset\n    * DatasetLoader\n    * Model\n    * ModelFactory\n    * Trainer\n    * Evaluator\n  * Split serving code into:\n    * Infrastructure\n    * Model (register, deploy)\n\n**Key Insights:**\n\n  * Modularize training and serving code for better organization and scalability.\n  * Utilize design patterns to enhance code maintainability and efficiency.\n\n\n\n**Tags:** Generative AI, LLMs, MLOps\n```'

In [16]:
answers = [prediction.outputs[0].text for prediction in predictions]
answers[0]

'```markdown\n# TL;DR Summary\n\n**Design Patterns:**\n\n  * Split training code into:\n    * Dataset\n    * DatasetLoader\n    * Model\n    * ModelFactory\n    * Trainer\n    * Evaluator\n  * Split serving code into:\n    * Infrastructure\n    * Model (register, deploy)\n\n**Key Insights:**\n\n  * Modularize training and serving code for better organization and scalability.\n  * Utilize design patterns to enhance code maintainability and efficiency.\n\n\n\n**Tags:** Generative AI, LLMs, MLOps\n```'

## Evaluate

The last step is to compute some metrics on the validation split to see how well our fine-tuned LLM performs.

In [17]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")


def compute_metrics(predictions: list[str], references: list[str]):
    result = rouge.compute(
        predictions=predictions, references=references, use_stemmer=True
    )
    result["mean_len"] = np.mean([len(p) for p in predictions])

    return {k: round(v, 4) for k, v in result.items()}

In [18]:
references = dataset["answer"]

In [19]:
references[0]

'```markdown\n# TL;DR Summary\n\n## Design Patterns\n- **Training Code Structure**: Key components include Dataset, DatasetLoader, Model, ModelFactory, Trainer, and Evaluator.\n- **Serving Code**: Infrastructure for Model registration and deployment.\n\n## Tags\n- Generative AI\n- LLMs\n```'

In [20]:
validation_metrics = compute_metrics(answers, references)
print(validation_metrics)

{'rouge1': 0.2208, 'rouge2': 0.1163, 'rougeL': 0.1801, 'rougeLsum': 0.1924, 'mean_len': 173.875}
